# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo
import datetime
from datetime import timedelta
import math
import pprint

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')


done


# Entrée 3 : période

In [3]:
"""
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')
"""

'\nT = int(input("Période (minutes) -> "))\ndelta = timedelta(minutes = float(T/2))\n\ndatemin = date - delta\ndatemax = date + delta\n\ndatemin = datetime.timestamp(datemin)\ndatemax = datetime.timestamp(datemax)\n\nprint(\'\ndone\')\n'

# QUERY 1 

In [4]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option, zone, periode, params):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query).distinct("asset_id"))
    dico['neighbors_id']=liste
    dico['density']=len(liste)
    dico['period']=[datemin, datemax]
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    dico['settings'] = params
    
    return dico


# QUERY 2

In [5]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.

def time_gap(current, new):

    td = abs(current - new)
    res = td/60
    return res


def asset_density(id_asset, rad, datemin, datemax, params):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    lon_average = 0
    lat_average = 0
    
    density = 0
    res = {}
    liste = []
    num = 0
    time = 0
    
    for doc in tmp:
        
        if time_gap(time, doc['recorded_at']) > 2:
            
            time = doc['recorded_at']
            
            lon = doc['location']['geo']['coordinates'][0]
            lat = doc['location']['geo']['coordinates'][1]         
            lon_average += lon
            lat_average += lat
            num += 1
            
            d_list = density_list(lon, lat, rad, datemin, datemax,
                                  {"asset_id": {"$ne": id_asset}}, True, True, params)

            if d_list['density'] > 0:

                for neighbor in d_list['neighbors_id']:

                    if neighbor not in liste:
                        density += 1
                        liste.append(neighbor)

    if density > 0:
        lon_average = lon_average/num
        lat_average = lat_average/num
        
        res['asset_id'] = id_asset
        res['density'] = density
        res['neighbors_id']= liste
        
        z = {}
        z['center'] = [lon_average, lat_average]
        z['radius'] = rad
        res['zone'] = z
        res['period'] = [datemin, datemax]
        res['settings'] = params
        
    return res


# QUERY 3

In [6]:
# QUERY 3
# Sur la base de tous les assets, retourner le Top – k [e.g. top 10] des zones les plus denses


def top_k_assets(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []
    now = datetime.datetime.now()

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax, {'function': 'top_k_assets', 'query_date' : now})

        if zone != {}:
            zones.append(zone)

    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


# Query4

In [7]:
import math
import datetime

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones(k, rad):
    
    now = datetime.datetime.now()
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            d = density_list(lon, lat, rad, 0, 0,{}, True, False, {'function': 'top_k_zones', 'query_date' : now})
            if d['density'] > 0:
                zones.append(d)
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


def top_k_periods(k, delta, option, params):
    
    # toutes les points de la base de données
    base = list(collection.find())
    base = sorted(base, key=lambda point:point['recorded_at'])
    print('sorting done\n')
    
    
    zones = []
    
    # intervalle de temps en heures converti en secondes
    T = delta
    delta = datetime.timedelta(hours=delta).total_seconds()
    
    end = base[len(base)-1]['recorded_at']
    datemin = base[0]['recorded_at']
    datemax = datemin + delta
    
    
    while datemin < end:
        
        d = density_list(-1, -1, -1, datemin, datemax, option, False, True, params)
        
        if d['density'] > 0:
            zones.append(d)
             
        datemin = datemax
        datemax = datemin + delta
        
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if  k == 'all':
        return zones
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


In [8]:
def top_k_zones_periods(k, rad, delta):
    
    now = datetime.datetime.now()
    param = {'function': 'top_k_zones_periods', 'query_date' : now, 'period': delta}
    
    periods = top_k_periods('all', delta, {}, {})
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
                        
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            for p in periods:
                
                if p['density'] < 20:
                    break
                    
                zones.append(density_list(lon, lat, rad, p['period'][0], p['period'][1], {}, True, True, param))
                
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


# Query 5

In [9]:
# QUERY 5
# Retournez le top k des périodes T où nous avons le plus de voitures qui sont en circulation

# les trajets (sans stay points)
op = {"moving": 1}

#zones = top_k_periods(k, delta, op, {'function': 'top_k_periods_moving', 'query_date' : now, 'delta': delta})
#pprint.pprint(zones)

print('\ndone')


done


# Appel aux fonctions

In [12]:

def enter_longitude():
    lon = float(input("longitude -> "))
    return lon
                      
def enter_latitude():        
    lat = float(input("latitude -> "))
    return lat
                
def enter_radius():
    rad = float(input("rayon (km) -> "))
    return rad
    
def enter_k():
    k = int(input("k -> "))
    return k
                
def enter_id():
    id_ = input("id asset -> ")
    return id_

def enter_date(label):
    
    print("\nDate", label)
    year = int(input("Annee -> "))
    month = int(input("Mois -> "))
    day = int(input("Jour -> "))
    hour = int(input("Heure -> "))
    minutes = int(input("Minutes -> "))
    seconds = int(input("Secondes -> "))
    date = datetime.datetime(year, month, day, hour, minutes, seconds)

    return datetime.datetime.timestamp(date)
                
def enter_period():
    T = float(input("Période (heures) -> "))
    return T


def calcul():
    
    go_on = True
    nums = [1, 2, 3, 4, 5, 6, 7]
    
    while go_on:

        print("(1) Calculer la densité d'une zone Z sur une période T"),
        print("(2) Calculer le nombre de points voisins d'un asset X sur une période T")
        print("(3) Top k des assets ayant le plus de voisins")
        print("(4) Top k des périodes les plus denses")
        print("(5) Top k des zones les plus denses")
        print("(6) Top k des périodes/zones les plus denses")
        print("(7) Top k des périodes où le plus de voitures sont en circulation")
        print("(0) Quitter\n")

        entree = int(input("Choisissez une fonction -> "))
        
        if entree == 0:
            break

        print()
        now = datetime.datetime.now()

        if entree == 1:
            lon = enter_longitude()
            lat = enter_latitude()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")


            param = {'function': 'density_list', 'query_date' : now}

            print("\n-------------------------")
            result = density_list(lon, lat, rad, d1, d2, {}, True, True, param)                 
            pprint.pprint(result)
            print("\n-------------------------")

        if entree == 2:
            id_ = enter_id()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")

            param = {'function': 'asset_density', 'query_date' : now}

            print("\n-------------------------")
            result = asset_density(id_, rad, d1, d2, param)
            pprint.pprint(result)
            print("\n-------------------------")

        if entree == 3:
            k = enter_k()
            rad = enter_radius()

            d1 = enter_date("début")
            d2 = enter_date("fin")

            print("\n-------------------------")
            result = top_k_assets(k, rad, d1, d2)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des périodes les plus denses
        if entree == 4:
            k = enter_k()
            T = enter_period()

            param = {'function': 'top_k_periods', 'query_date' : now, 'period': T}

            print("\n-------------------------")
            result = top_k_periods(k, T, {}, param)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des zones les plus denses\n",
        if entree == 5:
            k = enter_k()
            rad = enter_radius()

            print("\n-------------------------")
            result = top_k_zones(k, rad)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des zones/périodes les plus denses\n",
        if entree == 6:
            k = enter_k()
            rad = enter_radius()
            T = enter_period()

            print("\n-------------------------")
            result = top_k_zones_periods(k, rad, T)
            pprint.pprint(result)
            print("\n-------------------------")

        # top k des périodes où les voitures sont en circulation
        if entree == 7:
            k = enter_k()
            T = enter_period()

            # les trajets (sans stay points)
            op = {"moving": 1}
            param = {'function': 'top_k_periods_moving', 'query_date' : now, 'period': T}

            print("\n-------------------------")
            result = top_k_periods(k, T, op, param)
            pprint.pprint(result)
            print("\n-------------------------")
            
        if entree in nums:
            return result


In [13]:
res1 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 1

longitude -> -73.88
latitude -> 45
rayon (km) -> 10

Date début
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 15
Minutes -> 00
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 20
Minutes -> 00
Secondes -> 00

-------------------------
{'density': 0,
 'neighbors_id': [],
 'period': [1583244000.0, 1583262000.0],
 'settings': {'function': 'density_list',
              'query_date': datetime.datetime(2020, 8, 3, 9, 49, 28, 456127)},
 'zone': {'center': [-73.88, 45.0], 'radius': 10.0}}

-------------------------


In [14]:
res2 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 2

id asset -> 14256004559268
rayon (km) -> 10

Date début
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 15
Minutes -> 00
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 16
Minutes -> 00
Secondes -> 0

-------------------------
{'asset_id': '14256004559268',
 'density': 2,
 'neighbors_id': ['14256004535292', '14256004644482'],
 'period': [1583244000.0, 1583247600.0],
 'settings': {'function': 'asset_density',
              'query_date': datetime.datetime(2020, 8, 3, 9, 53, 40, 120218)},
 'zone': {'center': [-73.616475, 45.55815], 'radius': 10.0}}

-------------------------
done


In [16]:
res3 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 3

k -> 10
rayon (km) -> 10

Date début
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 12
Minutes -> 00
Secondes -> 0

Date fin
Annee -> 2020
Mois -> 3
Jour -> 3
Heure -> 12
Minutes -> 45
Secondes -> 00

-------------------------
[{'asset_id': '14429000029509',
  'density': 11,
  'neighbors_id': ['14429000004924',
                   '14429000035159',
                   '14429000043468',
                   '14429000044698',
                   '14429000051057',
                   '14429000054549',
                   '14429000058029',
                   '14429000059704',
                   '1442900005

In [17]:
res4 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 4

k -> 10
Période (heures) -> 5

-------------------------
sorting done

[{'density': 90,
  'neighbors_id': ['14256004530491',
                   '14256004535292',
                   '14256004559268',
                   '14256004569226',
                   '14256004574044',
                   '14256004592574',
                   '14256004664720',
                   '14295000006551',
                   '14295000011387',
                   '14295000017319',
                   '14429000003116',
                   '14429000004924',
                   '14429000005848',
                   '144290000

In [18]:
res5 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 5

k -> 10
rayon (km) -> 15

-------------------------
sorting done

[{'density': 41,
  'neighbors_id': ['14429000003116',
                   '14429000005848',
                   '14429000006838',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000010277',
                   '14429000014790',
                   '14429000015714',
                   '14429000016597',
                   '14429000026877',
                   '14429000030150',
                   '14429000032073',
                   '14429000034814

In [19]:
res6 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 6

k -> 10
rayon (km) -> 15
Période (heures) -> 7

-------------------------
sorting done

sorting done

[{'density': 34,
  'neighbors_id': ['14429000003116',
                   '14429000005848',
                   '14429000006838',
                   '14429000008883',
                   '14429000008909',
                   '14429000010095',
                   '14429000014790',
                   '14429000016597',
                   '14429000026877',
                   '14429000030150',
                   '14429000032073',
                   '14429000034814',
                   '14429000035217'

In [20]:
res7 = calcul()
print("done")

(1) Calculer la densité d'une zone Z sur une période T
(2) Calculer le nombre de points voisins d'un asset X sur une période T
(3) Top k des assets ayant le plus de voisins
(4) Top k des périodes les plus denses
(5) Top k des zones les plus denses
(6) Top k des périodes/zones les plus denses
(7) Top k des périodes où le plus de voitures sont en circulation
(0) Quitter

Choisissez une fonction -> 7

k -> 10
Période (heures) -> 5

-------------------------
sorting done

[{'density': 84,
  'neighbors_id': ['14256004535292',
                   '14256004559268',
                   '14256004569226',
                   '14256004574044',
                   '14256004664720',
                   '14295000006551',
                   '14429000003116',
                   '14429000004924',
                   '14429000005848',
                   '14429000006838',
                   '14429000008834',
                   '14429000008883',
                   '14429000008909',
                   '144290000